# Introduction

Provided the data downloaded from www.euets.info as zip file, we extract the data using the *ziploader* sub-module:

# Packages and options

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pyeutl.ziploader import (get_installations, get_accounts, get_transactions,
                              get_compliance, get_account_holders) 

from pyeutl.ziploader.utils import (load_zipped_file, map_if_exists, get_mapper)
from pyeutl.ziploader.category_mappings import (map_activity_category, map_account_category,
                              map_nace_category)
%load_ext autoreload
%autoreload 2

In [2]:
fn_zip = "./eutl_2023.zip"

# Get Data

Get installation data from zip file. The function merges information from lookup tables into the installation table:

In [3]:
df_inst = get_installations(fn_zip, )
df_inst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17975 entries, 0 to 17974
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   17975 non-null  object 
 1   name                 17975 non-null  object 
 2   tradingSystem_id     17975 non-null  object 
 3   registry_id          17975 non-null  object 
 4   activity_id          17975 non-null  int64  
 5   eprtrID              5420 non-null   object 
 6   parentCompany        6471 non-null   object 
 7   subsidiaryCompany    3780 non-null   object 
 8   permitID             16310 non-null  object 
 9   designatorICAO       630 non-null    object 
 10  monitoringID         1636 non-null   object 
 11  monitoringExpiry     0 non-null      float64
 12  monitoringFirstYear  1637 non-null   object 
 13  permitDateExpiry     5088 non-null   object 
 14  isAircraftOperator   17947 non-null  object 
 15  ec748_2009Code       1636 non-null  

Get account holder informations

In [4]:
df_accountHolder = get_account_holders(fn_zip)
df_accountHolder = df_accountHolder.rename(columns = {
    c: "accountHolder" + c.capitalize() for c in df_accountHolder.columns if c!= "id"})
df_accountHolder.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18238 entries, 0 to 18237
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   id                                  18238 non-null  int64 
 1   accountHolderTradingsystem_id       18238 non-null  object
 2   accountHolderName                   18212 non-null  object
 3   accountHolderAddressmain            18151 non-null  object
 4   accountHolderPostalcode             18082 non-null  object
 5   accountHolderCity                   18168 non-null  object
 6   accountHolderLegalentityidentifier  2334 non-null   object
 7   accountHolderCountry_id             18128 non-null  object
 8   accountHolderCountry                18128 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.3+ MB


Next, load the account data. The account dataframe includes the information of related look-up tables. If a dataframe with installation or account holder information is passed to the function, this information is also merged into the account table:

In [5]:
df_acc = get_accounts(fn_zip, 
                  df_installation=df_inst[["id", "name", "activity_id", "activity", "activityCategory", 
                                                   "nace_id", "nace", "naceCategory"]],
                    df_accountHolder=df_accountHolder)
df_acc.info()
# export the list with all accounts and related installation and account holder informations
df_acc.to_excel("eutl_account_full_list.xlsx", index=False)

e:\GIT\pyeutl\venv\lib\site-packages\pandas\core\reshape\merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


<class 'pandas.core.frame.DataFrame'>
Int64Index: 43018 entries, 0 to 43017
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  43018 non-null  int64  
 1   tradingSystem_id                    42153 non-null  object 
 2   accountIDESD                        226 non-null    object 
 3   yearValid                           224 non-null    float64
 4   name                                42146 non-null  object 
 5   registry_id                         43018 non-null  object 
 6   accountHolder_id                    42149 non-null  float64
 7   accountType_id                      42126 non-null  object 
 8   isOpen                              42153 non-null  object 
 9   openingDate                         41891 non-null  object 
 10  closingDate                         26706 non-null  object 
 11  commitmentPeriod                    40921

Compliance data follow the same logic:

In [6]:
df_comp = get_compliance(fn_zip, df_installation=df_inst)
df_comp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 481762 entries, 0 to 481761
Data columns (total 49 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    481762 non-null  object 
 1   installation_id       481762 non-null  object 
 2   year                  481762 non-null  int64  
 3   reportedInSystem_id   481762 non-null  object 
 4   euetsPhase            481538 non-null  object 
 5   compliance_id         209237 non-null  object 
 6   allocatedFree         224431 non-null  float64
 7   allocatedNewEntrance  6899 non-null    float64
 8   allocatedTotal        481762 non-null  int64  
 9   allocated10c          1548 non-null    float64
 10  verified              200025 non-null  float64
 11  verifiedCummulative   210478 non-null  float64
 12  verifiedUpdated       481538 non-null  object 
 13  surrendered           182958 non-null  float64
 14  balance               224 non-null     float64
 15  

Transactions follow the same logic also providing the descriptions provided in the look-up tables. In addition, we can pass a table with account information. The transaction loader also allows to resample the transaction data to a higher frequency: 

In [7]:
freq = "M"
# get transactions merged with account information
cols = ['id', 'name', 'registry_id', 'accountHolder_id', 'accountType_id',
        'isRegisteredEutl', 'installation_id', 'registry', 'accountType', 
        'accountCategory', 'installationName', 'activity_id', 'activity', 
        'nace_id', 'nace', "naceCategory" ]
df_trans = get_transactions(fn_zip, drop=["project_id"], freq=freq, 
                      df_account=df_acc[cols])
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997164 entries, 0 to 1997163
Data columns (total 45 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   date                             datetime64[ns]
 1   id                               int64         
 2   transactionID                    object        
 3   transactionTypeMain_id           int64         
 4   transactionTypeSupplementary_id  int64         
 5   transferringAccount_id           float64       
 6   acquiringAccount_id              float64       
 7   unitType_id                      object        
 8   tradingSystem_id                 object        
 9   acquiringYear                    float64       
 10  transferringYear                 float64       
 11  amount                           int64         
 12  transactionTypeMain              object        
 13  transactionTypeSupplementary     object        
 14  unitType                         o

We can wrap all of this into a single function, e.g.:

In [8]:
def load_data(fn_zip, freq=None):
    """Load installation account and transaction data from zip file
    :param fn_zip: <string> name of zip file with eutl data
    :param freq: <string> frequency for resampling of transaction data"""
    # get installations
    df_inst = get_installations(fn_zip)
    # get accounts with selected installation information
    df_acc = get_accounts(fn_zip, 
                          df_installation=df_inst[["id", "name", "activity_id", "activity", "activityCategory", 
                                                   "nace_id", "nace", "naceCategory"]]) 
    # get compliance data with installation information
    df_comp = get_compliance(fn_zip, df_installation=df_inst)    
    # get transactions merged with account information
    cols = ['id', 'name', 'registry_id', 'accountHolder_id', 'accountType_id',
            'isRegisteredEutl', 'installation_id', 'registry', 'accountType', 
            'accountCategory', 'installationName', 'activity_id', 'activity', 
            'nace_id', 'nace', "naceCategory" ]
    df_trans = get_transactions(fn_zip, drop=["project_id"], freq=freq, 
                          df_account=df_acc[cols])
    return df_inst, df_acc, df_comp, df_trans

df_inst, df_acc, df_comp, df_trans = load_data(fn_zip, freq=None)